# Excercise 3: Track Experiments

## Prepare data once again

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

start_df = pd.read_csv('./clean-loan-data.csv', low_memory=False)

pd.set_option('display.max_columns', None)

df = start_df.copy(deep=True)
df['TARGET'] = [0 if i=='Default' else 1 for i in df['loan_status']]
df.drop(df.columns.difference(['loan_amnt', 'annual_inc', 'TARGET']), 1, inplace=True)
df = df.dropna()
df = pd.get_dummies(df)

In [2]:
from sklearn.model_selection import train_test_split
X_train = df.drop('TARGET',axis=1)
y_train = df['TARGET']
X_train, X_test, y_train, y_test = train_test_split(df.drop('TARGET',axis=1),df['TARGET'],test_size=0.15,random_state=101,  stratify=df['TARGET'])

In [3]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=12)
x_train_r, y_train_r = sm.fit_resample(X_train, y_train)

# Excercise 1: Track Experiment

Check the documentation at https://www.mlflow.org/docs/latest/tracking.html#logging-data-to-runs and track the experiment using these functions.

1. set_experiment
2. start run
3. log_param
4. log_metric
5. set_tag

Check the MLFlow UI: http://localhost:5500

*Questions*
- Experiment with these values. How would you use them?
- Can your reproduce how the pipeline looks in the MLFlow UI?

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline

import mlflow


mlflow.set_tracking_uri("http://mlflow:5500")

    

In [17]:
mlflow.set_experiment("/test-exp")
with mlflow.start_run():
    log_reg = LogisticRegression(C = 0.0001,random_state=12)
    log_reg.fit(x_train_r, y_train_r)
    mlflow.log_param("C", 0.0001)
    mlflow.log_param("random_state", 12)
    mlflow.log_metric("random_state", 200)
    confusion_matrix(y_test, log_reg.predict(X_test))
    mlflow.set_tag( "hello", "world")

## Excercise 2: Autolog

Instead of tracking everything manually, MLFlow supports autotracking of several libraries including sklearn.

So reuse your code above, remove all calls to `set_tag` and `log_param`. Instead call `mlflow.autolog()` before starting the run.

Check the MLFlow UI: http://localhost:5500

Questions
- How did that change in the MLFlow UI

In [18]:
import mlflow


mlflow.set_tracking_uri("http://mlflow:5500")


In [20]:
mlflow.set_experiment("/exercise-2")
mlflow.autolog()
with mlflow.start_run():
    log_reg = LogisticRegression(C = 0.0001,random_state=12)
    log_reg.fit(x_train_r, y_train_r)
    confusion_matrix(y_test, log_reg.predict(X_test))

2023/06/10 12:43:46 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2023/06/10 12:43:46 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2023/06/10 12:43:46 INFO mlflow.utils.autologging_utils: sklearn autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow to the MLflow run with ID '0f30d030c9914933a700d0f51b86cf0e'


INFO: '/exercise-2' does not exist. Creating a new experiment


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function plot_confusion_matrix is deprecated; Function `plot_confusion_matrix` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: ConfusionMatrixDisplay.from_predictions or ConfusionMatrixDisplay.from_estimator.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function plot_roc_curve is deprecated; Function :func:`plot_roc_curve` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: :meth:`sklearn.metric.RocCurveDisplay.from_predictions` or :meth:`sklearn.metric.RocCurveDisplay.from_estimator`.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function plot_precision_recall_curve is deprecated; Function `plot_precision_recall_curve` is deprecated in 1.0 and will be removed in 1.2. Use o